# Tort Law Experiment

In [18]:
from tort_dataset import *
from neural_networks import *

from tqdm import tqdm
from joblib import Parallel, delayed

%load_ext autoreload
%autoreload 2

# Increase resolution of plots
plt.rcParams['figure.dpi'] = 150

# Show more columns
pd.set_option('display.max_column',None)

### Initialize global parameters

In [21]:
# Datasets parameters
db_size = 5000
db_size_small = 500

# Network parameters
hidden_layers = [(12), (24, 6), (24,10,3)]
activation = 'logistic'
max_iter = 50000
learning_rate_init = 0.001
solver = 'adam'
batch_size = 50

### Intialize datasets

In [20]:
# Unique dataset
unique_df = generate_unique_dataset()
X_unique, y_unique = preprocess(unique_df)

# Regular training dataset
train_df = generate_dataset(db_size)
X_train, y_train = preprocess(train_df)

# Regular test dataset
test_df = generate_dataset(db_size)
X_test, y_test = preprocess(test_df)

# Smaller training dataset
small_train_df = generate_dataset(db_size_small)
X_train_small, y_train_small = preprocess(small_train_df)

# Special test datasets to test the learned rationale (unlawfulness and impudence)
unl_df = generate_unlawful_dataset()
X_unl, y_unl = preprocess(unl_df)

imp_df = generate_impudence_dataset()
X_imp, y_imp = preprocess(imp_df)

### Train networks

In [22]:
def create_network(layers):
    '''Creates an MLP with a given number of hidden layers'''
    return MLPClassifier(activation=activation, hidden_layer_sizes=layers, 
                         max_iter=max_iter, learning_rate_init=learning_rate_init,
                         solver=solver, batch_size=batch_size)

def train_networks(nn, X, y):
    nn.fit(X, y)
    return nn

In [23]:
# Create and train the neural networks on the regular training set
neural_nets = [create_network(hls) for hls in hidden_layers]   
neural_nets = Parallel(n_jobs=-1)(delayed(train_networks)(nn, X_train, y_train) 
                                 for nn in tqdm(neural_nets))

# # Create and train the neural networks on the smaller training set
neural_nets_small = [create_network(hls) for hls in hidden_layers]
neural_nets_small = Parallel(n_jobs=-1)(delayed(train_networks)(nn, X_train_small, y_train_small) 
                                 for nn in tqdm(neural_nets_small))

100%|██████████| 3/3 [00:00<00:00, 3224.73it/s]


## Test performance (accuracies)

In [26]:
# Store the predictions of the networks in their respective dataframes
for idx, (nn, nn_small) in enumerate(zip(neural_nets, neural_nets_small)):
    for df, X in zip([test_df, unique_df, unl_df, imp_df],
                     [X_test, X_unique, X_unl, X_imp]):
        
        #Predictions
        df['prediction_'+str(idx+1)] = nn.predict(X)
        df['prediction_small_'+str(idx+1)] = nn_small.predict(X)
        
        # probabilities
        df['proba_'+str(idx+1)] = nn.predict_proba(X)[:,1]
        df['proba_small_'+str(idx+1)] = nn_small.predict_proba(X)[:,1]
        
        #Add unl and imp
        df['unl'] = [int(is_unlawful(row)) for index, row in df.iterrows()]
        df['imp'] = [int(is_imputable(row)) for index, row in df.iterrows()]

In [24]:
# Regular training set 
accs = pd.DataFrame.from_dict({'regular '+str(idx+1) + ' layers': {
    'test': round(100*accuracy_score(y_test, nn.predict(X_test)), 2),
    'unique': round(100*accuracy_score(y_unique, nn.predict(X_unique)), 2),
    'unl': round(100*accuracy_score(y_unl, nn.predict(X_unl)), 2),
    'imp': round(100*accuracy_score(y_imp, nn.predict(X_imp)), 2),
} for idx, nn in enumerate(neural_nets)}, orient='index')
accs.to_csv('results/accuracies/accuracies_tort.csv')
accs

,test,unique,unl,imp
regular 1 layers,100.00,100.00,100.0,100.00
regular 2 layers,100.00,100.00,100.0,100.00
regular 3 layers,98.96,98.14,100.0,96.09


In [25]:
# smaller training set 
accs_smaller = pd.DataFrame.from_dict({'smaller '+str(idx+1) + ' layers': {
    'test': round(100*accuracy_score(y_test, nn.predict(X_test)), 2),
    'unique': round(100*accuracy_score(y_unique, nn.predict(X_unique)), 2),
    'unl': round(100*accuracy_score(y_unl, nn.predict(X_unl)), 2),
    'imp': round(100*accuracy_score(y_imp, nn.predict(X_imp)), 2),
} for idx, nn in enumerate(neural_nets_small)}, orient='index')
accs_smaller.to_csv('results/accuracies/accuracies_tort_smaller.csv')
accs_smaller

,test,unique,unl,imp
smaller 1 layers,98.42,97.07,90.48,92.97
smaller 2 layers,98.86,98.05,95.24,96.88
smaller 3 layers,98.24,96.97,92.86,94.53


## Test the Rationale

In [27]:
pd.DataFrame(imp_df['prediction_3'].groupby(by=imp_df['imp']).mean())

,prediction_3
imp,
0,0.3125
1,1.0000


In [28]:
pd.DataFrame(unl_df['prediction_3'].groupby(by=unl_df['unl']).mean())

,prediction_3
unl,
0,0
1,1


### Smaller dataset

In [29]:
pd.DataFrame(imp_df['prediction_small_3'].groupby(by=imp_df['imp']).mean())

,prediction_small_3
imp,
0,0.4375
1,1.0000


In [30]:
pd.DataFrame(unl_df['prediction_small_3'].groupby(by=unl_df['unl']).mean())

,prediction_small_3
unl,
0,0.214286
1,1.000000


## Run the experiment multiple times

In [15]:
import copy 

def convert_accs(accs):
    '''
    Converts list of accuracies to mean and std
    '''
    return str(round(np.mean(accs), 2)) + ' ± ' + str(round(np.std(accs), 2))   

def merge_accuracies(accuracies):
    '''
    Merge the list of accuracies to a pandas dataframe with mean and standard deviations
    '''
    accs = copy.deepcopy(accuracies)
    results = accs[0]
    for idx, x in enumerate(accs):
        if idx == 0: continue
        for train_name, train_data in x.items():
            for test_name, test_data in train_data.items():
                results[train_name][test_name].append(test_data[0])
                if idx == len(accs)-1:
                    results[train_name][test_name] = convert_accs(results[train_name][test_name])
    return pd.DataFrame.from_dict(results).T

def run_experiment():
    '''
    Train and test neural networks
    '''

    accuracies = {}

    # Training datasets
    datasets_train = {
        'regular' : generate_dataset(5000),
        'smaller' : generate_dataset(500),
    }
    preprocessed_train = {name: preprocess(df) for name, df in datasets_train.items()}

    # Test datasets
    datasets_test = {
        'general' : generate_dataset(5000),
        'unique' : generate_unique_dataset(),
        'unl' : generate_unlawful_dataset(),
        'imp' : generate_impudence_dataset(),
    }
    preprocessed_test = {name: preprocess(df) for name, df in datasets_test.items()}
    
    for train_name, train_data in preprocessed_train.items():       
        # Create and train the neural networks on training set A
        neural_nets = [create_network(hls) for hls in hidden_layers]   
        neural_nets = Parallel(n_jobs=1)(delayed(train_networks)(nn, train_data[0], train_data[1]) 
                                         for nn in neural_nets)
        
        # Initialize accuracies data
        for idx, nn in enumerate(neural_nets):
            accuracies[train_name+'_'+str(idx+1)] = {test_name: [] for test_name in preprocessed_test.keys()}

        # Add accuracies
        for test_name, test_data in preprocessed_test.items():
            for idx, nn in enumerate(neural_nets):
                accuracies[train_name+'_'+str(idx+1)][test_name].append(100*accuracy_score(test_data[1], nn.predict(test_data[0])))

    return accuracies

In [16]:
# Number of runs
num_runs = 50
accuracies = Parallel(n_jobs=-1)(delayed(run_experiment)() for run in tqdm(range(0, num_runs)))
results = merge_accuracies(accuracies)
results.to_csv('results/accuracies/50_runs_tort.csv')
results

100%|██████████| 50/50 [00:37<00:00,  1.32it/s]


,general,unique,unl,imp
regular_1,100.0 ± 0.0,100.0 ± 0.0,100.0 ± 0.0,100.0 ± 0.0
regular_2,100.0 ± 0.0,100.0 ± 0.0,100.0 ± 0.0,100.0 ± 0.0
regular_3,99.88 ± 0.33,99.79 ± 0.58,100.0 ± 0.0,99.67 ± 1.24
smaller_1,98.54 ± 0.49,97.41 ± 0.88,92.56 ± 3.43,92.14 ± 3.66
smaller_2,99.18 ± 0.41,98.56 ± 0.72,95.81 ± 3.25,96.19 ± 3.29
smaller_3,98.12 ± 0.69,96.67 ± 1.24,91.55 ± 5.96,91.94 ± 4.12
